<a href="https://www.kaggle.com/code/srsses/pyspark-analysis?scriptVersionId=156824020" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=6e4f597beccd1c4ee10731049d80b7c3fcbf4edccc6cfff2f15e5c1b7cbe870e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import pyspark.pandas as pyd
from pyspark.sql.functions import *
from pyspark.sql.types import *

/opt/conda/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
spark = SparkSession.builder.master("local").appName("analysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 09:31:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = pd.read_excel("/kaggle/input/trending-fifa-players-dataset/trending_football_players.xlsx")

In [5]:
df = spark.createDataFrame(df)

In [6]:
df.take(6)

[Row(Player_name='T. Almada', Images='https://cdn.sofifa.net/players/245/371/24_60.png', Age=22, National_team='Argentina', Positions="['CAM', 'CM', 'CF']", Overall=79, Potential_overall=87, Current_club='Atlanta United', Current_contract='2022 ~ 2025', Value='€39.5M', Wage='€10K', Total_stats=2050),
 Row(Player_name='L. Palma', Images='https://cdn.sofifa.net/player_0.svg', Age=23, National_team='Honduras', Positions="['LW']", Overall=69, Potential_overall=75, Current_club='Celtic', Current_contract='2023 ~ 2028', Value='€2.2M', Wage='€22K', Total_stats=1794),
 Row(Player_name='R. Lavia', Images='https://cdn.sofifa.net/players/263/620/24_60.png', Age=19, National_team='Belgium', Positions="['CDM']", Overall=73, Potential_overall=86, Current_club='Chelsea', Current_contract='2023 ~ 2030', Value='€7M', Wage='€32K', Total_stats=1829),
 Row(Player_name='W. Zaïre-Emery', Images='https://cdn.sofifa.net/players/270/673/24_60.png', Age=17, National_team='France', Positions="['CM', 'CDM']", Ove

In [7]:
df.select('Value').show()

+------+
| Value|
+------+
|€39.5M|
| €2.2M|
|   €7M|
|  €24M|
|€31.5M|
| €1.5M|
|  €46M|
| €9.5M|
|€25.5M|
| €112M|
|  €22M|
| €109M|
| €3.7M|
|  €15M|
|€41.5M|
|  €43M|
| €9.5M|
|  €10M|
|€83.5M|
|€30.5M|
+------+
only showing top 20 rows



In [8]:
#df.describe().show()

In [9]:
df.groupBy('Player_name').agg({'Overall':'SUM'}).sort('Player_name').show()

+--------------------+------------+
|         Player_name|sum(Overall)|
+--------------------+------------+
|      08\n L. Thuram|          83|
|        11\n Ronaldo|         154|
|          14\n Pinto|          72|
|          14\n Puyol|         166|
|     15\n R. Drenthe|          71|
|    16\n A. Arshavin|         150|
|       16\n Anderson|          73|
|     16\n FakePlayer|          69|
|       17\n F. Totti|         160|
|    17\n Xabi Alonso|         246|
|    18\n L. Pionnier|         146|
|19\n Fernando Torres|          77|
|          19\n Ganso|          75|
|        19\n Robinho|         154|
|       19\n Y. Touré|         160|
|       20\n B. Sagna|          71|
|    20\n David Villa|         158|
|    20\n Lucas Silva|          74|
|   21\n Alan Patrick|          72|
|     21\n David Luiz|          79|
+--------------------+------------+
only showing top 20 rows



In [10]:
@pandas_udf(StringType())
def convert(f):
    def convert_num(y):
        try:
            if y.endswith('K')==True:
                y = list(y)
                y.remove(y[''.join(y).find('K')])
                if ''.join(y).startswith('€')==True:
                    y.remove(y[''.join(y).find('€')])
                else:
                    pass
                try :
                    return str(int(''.join(y))*1000)
                except:
                    return y
            elif y.endswith('M')==True:
                y = list(y)
                y.remove(y[''.join(y).find('M')])
                if ''.join(y).startswith('€')==True:
                    y = list(y)
                    y.remove(y[''.join(y).find('€')])
                else:
                    pass
                try :
                    return str(float(''.join(y))*1000000)
                except:
                    return y
        except:
            return y
    return f.apply(convert_num)

In [11]:
df1 = df.withColumns({'Converted_value':convert('Value').cast(IntegerType()),'Converted_wage':convert('Wage').cast(IntegerType())})

In [12]:
df1.select('Value','Wage','Converted_value','Converted_wage').show()

+------+-----+---------------+--------------+
| Value| Wage|Converted_value|Converted_wage|
+------+-----+---------------+--------------+
|€39.5M| €10K|       39500000|         10000|
| €2.2M| €22K|        2200000|         22000|
|   €7M| €32K|        7000000|         32000|
|  €24M|  €9K|       24000000|          9000|
|€31.5M| €28K|       31500000|         28000|
| €1.5M|  €1K|        1500000|          1000|
|  €46M|€110K|       46000000|        110000|
| €9.5M|  €7K|        9500000|          7000|
|€25.5M| €77K|       25500000|         77000|
| €112M|€190K|      112000000|        190000|
|  €22M| €41K|       22000000|         41000|
| €109M| €95K|      109000000|         95000|
| €3.7M| €500|        3700000|          NULL|
|  €15M|  €7K|       15000000|          7000|
|€41.5M| €40K|       41500000|         40000|
|  €43M|€110K|       43000000|        110000|
| €9.5M| €21K|        9500000|         21000|
|  €10M| €26K|       10000000|         26000|
|€83.5M| €65K|       83500000|    

In [13]:
df.select('Positions').show()

+-------------------+
|          Positions|
+-------------------+
|['CAM', 'CM', 'CF']|
|             ['LW']|
|            ['CDM']|
|      ['CM', 'CDM']|
|      ['CM', 'CAM']|
|      ['CAM', 'CM']|
|['CAM', 'RW', 'ST']|
|      ['CDM', 'CM']|
|             ['ST']|
|      ['CAM', 'CM']|
|      ['CAM', 'CM']|
|       ['LW', 'RW']|
|       ['CB', 'LB']|
|             ['CB']|
|['RW', 'ST', 'CAM']|
|      ['CDM', 'CM']|
|      ['RB', 'CDM']|
|             ['ST']|
|      ['CAM', 'CM']|
|['RB', 'RWB', 'CB']|
+-------------------+
only showing top 20 rows



In [14]:
@pandas_udf(ArrayType(StringType()))
def explodes(f):
    return f.apply(eval)

In [15]:
df.select('Positions',(explode(explodes(df.Positions))).alias('exploded_positions')).show()

+-------------------+------------------+
|          Positions|exploded_positions|
+-------------------+------------------+
|['CAM', 'CM', 'CF']|               CAM|
|['CAM', 'CM', 'CF']|                CM|
|['CAM', 'CM', 'CF']|                CF|
|             ['LW']|                LW|
|            ['CDM']|               CDM|
|      ['CM', 'CDM']|                CM|
|      ['CM', 'CDM']|               CDM|
|      ['CM', 'CAM']|                CM|
|      ['CM', 'CAM']|               CAM|
|      ['CAM', 'CM']|               CAM|
|      ['CAM', 'CM']|                CM|
|['CAM', 'RW', 'ST']|               CAM|
|['CAM', 'RW', 'ST']|                RW|
|['CAM', 'RW', 'ST']|                ST|
|      ['CDM', 'CM']|               CDM|
|      ['CDM', 'CM']|                CM|
|             ['ST']|                ST|
|      ['CAM', 'CM']|               CAM|
|      ['CAM', 'CM']|                CM|
|      ['CAM', 'CM']|               CAM|
+-------------------+------------------+
only showing top